In [86]:
import jieba
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from os import path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [87]:
Q5_data = pd.read_excel('train.xlsx')
Q5_data

,target,comment
0,1,订餐 半小时 不见 餐 催 挂 电话
1,1,太 不靠 谱 漏发 订单 菜 送错 早 送 小时
2,0,超级 超级 喜欢 吃 每次 好开心 口味 建议 手套 戴 几双 吃 完 形象...
3,0,谢谢 快递 哥哥 准时 辛苦
4,1,差得 餐厅 上菜 慢 贵 餐厅 饿死 那种 面条 味道 一碗 ...
...,...,...
17948,0,感觉 好像 发挥 稳定 几此 有好有坏 吃 日料 关注 三文鱼 中等 不...
17949,1,订 九点 送到 十二点
17950,1,差评 辣白菜 五花肉 石锅 拌 饭 超级 难吃 五花肉 硬 难吃 感觉 新鲜 ...
17951,1,贵 好吃 老婆 几次 非要 一吃 真 难吃 蒸 没错 炒菜 油腻 菜 好...


In [88]:
raw_df = pd.read_excel('train.xlsx')
stop_words = set(open('ChineseStopWords.txt', encoding='utf-8').read().strip().split('\n'))
 
# 分词，生成新的dataframe
data = []  # 存储分词后的数据和label
for comment,target in raw_df.values:
    words = [
        # 过滤停用词
        word for word in jieba.cut(str(comment)) if word.strip() and word.strip() not in stop_words
    ]
    data.append([' '.join(words), target])
 
df = pd.DataFrame(data, columns=['label', 'cut'])
print(df)

      label                                                cut
0         1                             订餐   半小时 不见 餐   催 挂 电话
1         1               太 不靠 谱           漏发 订单 菜 送错   早 送 小时
2         0  超级 超级 喜欢 吃   每次 好开心   口味   建议 手套 戴 几双   吃 完 形象...
3         0                                 谢谢 快递 哥哥   准时   辛苦
4         1  差得 餐厅   上菜 慢   贵   餐厅 饿死 那种   面条 味道   一碗      ...
...     ...                                                ...
17948     0  感觉 好像 发挥 稳定   几此   有好有坏   吃 日料 关注 三文鱼   中等   不...
17949     1                                    订 九点 送到   十二点  
17950     1  差评   辣白菜 五花肉 石锅 拌 饭 超级 难吃   五花肉 硬 难吃   感觉 新鲜  ...
17951     1  贵 好吃   老婆 几次 非要   一吃 真 难吃   蒸 没错   炒菜 油腻   菜 好...
17952     1              小炒 肉太辣   辣椒 放太多   凉粉 更辣   筷子 要用 手 抓 吃

[17953 rows x 2 columns]


In [89]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2))  
X = vectorizer.fit_transform(df['cut'].astype(str).values)  
print(type(X))  
X.shape  
# 打印最后10个特征词
print(vectorizer.get_feature_names()[-10:])

<class 'scipy.sparse._csr.csr_matrix'>
['龙鱼 味道', '龟苓膏', '龟苓膏 味道', '龟苓膏 总体', '龟苓膏 特别', '龟速', '龟速 下次', '龟速 十二点', '龟龄膏', '龟龄膏 正宗']


In [90]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

selector = SelectKBest(chi2, k=50)
new_X = selector.fit_transform(X, df['label'])

import numpy as np

np.array(vectorizer.get_feature_names())[selector.get_support()]

array(['一个半', '一个半 小时', '不好', '不错', '不错 价格', '不错 喜欢', '两个 小时', '他家', '价格',
       '再也', '再也不会', '分钟', '味道', '味道 不错', '喜欢', '垃圾', '太差', '太慢', '失望',
       '好吃', '好喝', '实惠', '小哥', '小时', '小时 送到', '差差', '差评', '很差', '很快',
       '恶心', '打电话', '挺不错', '朋友', '每次', '没法', '热情', '环境', '环境 不错', '电话',
       '真的', '米饭', '美味', '订单', '辛苦', '还会', '送到', '送来', '适合', '难吃', '面包'],
      dtype='<U11')

In [91]:
from sklearn.feature_extraction.text import TfidfTransformer
weight_X = TfidfTransformer().fit_transform(new_X)
print(weight_X)

  (0, 38)	1.0
  (1, 42)	0.8246362152349382
  (1, 23)	0.5656634268918193
  (2, 33)	0.7930202251233012
  (2, 14)	0.6091953073895011
  (3, 43)	1.0
  (4, 12)	1.0
  (5, 33)	1.0
  (6, 39)	0.35414520220151335
  (6, 30)	0.5758258711206533
  (6, 23)	0.42688805004738406
  (6, 6)	0.6006432673663562
  (10, 21)	0.6707918241606595
  (10, 13)	0.5349074204897583
  (10, 12)	0.360447904100054
  (10, 3)	0.36604601974900086
  (11, 2)	1.0
  (16, 12)	1.0
  (18, 14)	0.5936373413129162
  (18, 7)	0.8047326928856141
  (20, 39)	0.8981255383593754
  (20, 35)	0.39847073600517696
  (20, 12)	0.18598814449898388
  (21, 19)	1.0
  (22, 28)	0.8602977758230352
  :	:
  (17932, 11)	1.0
  (17933, 36)	1.0
  (17934, 15)	0.7021838865021887
  (17934, 0)	0.7119956387062223
  (17935, 36)	0.8055726966861207
  (17935, 12)	0.5924969454384145
  (17936, 48)	1.0
  (17937, 14)	1.0
  (17938, 39)	0.9049925030438183
  (17938, 33)	0.42542751372529325
  (17941, 23)	0.47881398637719963
  (17941, 17)	0.5956901030695095
  (17941, 11)	0.64489570

In [1]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(weight_X, df['label'], test_size=0.3)  # 训练集测试集7 3开

print('train data: ', train_X.shape, train_y.shape)  # 打印训练集shape
print('test data: ', test_X.shape, test_y.shape)  # 打印测试集shape

NameError: name 'weight_X' is not defined

In [ ]:
# 使用简单神经网络模型
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(train_X,train_y)


In [93]:
# 使用逻辑回归模型
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(train_X, train_y) 

In [94]:
clf.score(test_X, test_y)  

0.779428147047902

In [114]:
clf = LogisticRegression(C=10, max_iter=1000).fit(train_X, train_y)  # 声明某些参数
print('train acc: ', clf.score(train_X, train_y))  # 评估训练集acc
print('test acc: ', clf.score(test_X, test_y))  # 评估训练集acc

train acc:  0.7853107344632768
test acc:  0.7799851466765689


In [96]:
import pickle
import re
pickle.dump(clf, open('./Q5/lr.model', 'wb'))
data = pd.read_excel('./Q5/test.xlsx')
data

,target,userId,sellerId,timestamp,comment
0,NaN,72916,11976,2020-10-01 12:38:17,text：买的人很多，味道好，是十年以上的老店了~~~~\n
1,NaN,39091,34918,2020-10-01 17:35:04,原来是辣的流泪，所以叫伤心凉粉。厉害，味道不错，不过吃完还想吃。\n
2,NaN,289933,59229,2020-10-01 21:00:17,text：外卖好快的，适合中午点
3,NaN,77764,27100,2020-10-02 06:40:07,text：整体感觉就是一般，味道也一般，可能是个人因素哈．\n
4,NaN,24687,30939,2020-10-02 07:28:33,text：我不喜欢
...,...,...,...,...,...
1495,NaN,279254,54944,2021-08-01 08:15:56,text：感觉价钱太贵了。
1496,NaN,18389,59255,2021-08-01 08:41:58,text：鸭血粉丝不正宗，鸭血不嫩汤也没味道，桂花凉粉还凑合\n
1497,NaN,1901,31488,2021-07-31 14:23:52,text：外卖天天有红包\n
1498,NaN,542,58869,2021-07-31 17:06:09,text：适合羊毛党，折扣多


In [97]:
def clean_data(x):
    text = re.sub(r"[A-Za-z0-9(),!?\'\'\.【】⊙﹏…ฅω*ฅ❀༵〜⚘;%^:#＆@？！‘’：_，“”、-。~～·\.-×☆]"," ",x)
    return text.strip()
data['comment']=data['comment'].apply(lambda x:clean_data(x).replace('\n',''))

In [98]:
stopword=[line.strip() for line in open('ChineseStopWords.txt','r',encoding="utf-8").readlines()]
def cut_word(text):
    try:
        clean_text=[]
        for i in jieba.lcut(text):
            if i:
                if i not in stopword and i:
                    clean_text.append(i)
        return ''.join(clean_text)
    except:
        print(text)
data['comment']=data['comment'].apply(lambda x:cut_word(x))

In [99]:
data.to_excel('./Q5/clean_test.xlsx',index=None)

In [155]:
data = pd.read_excel('./Q5/clean_test.xlsx')
vectorizer = CountVectorizer(ngram_range=(1, 2))  
testX = vectorizer.fit(data['comment'].astype(str).values)

AttributeError: 'CountVectorizer' object has no attribute 'isnull'

In [154]:
# 加载模型并预测
clf = pickle.load(open('./Q5/lr.model', 'rb'))
data['target'] = clf.predict(testX)
data

ValueError: Expected 2D array, got scalar array instead:
array=CountVectorizer(ngram_range=(1, 2)).
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
data.to_excle('./Q5/pretest.xlsx',index=None)